# 02.A: Working with Datasets

It's important to have a clear and sensible way of representing the datasets that learning algorithms train on. A dataset consists of $n$ examples. Each example consists of $m$ features. This makes $m$ the number dimensions the dataset has. In supervised learning, the dataset is a matrix like this:

$\boldsymbol{D} =\left[\begin{array}{cccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)} & y^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)} & y^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)} & y^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)} & y^{(n)}
\end{array}\right]$

Each row of this matrix is an example consisting of the $m$ features plus the target label as the last element in the row. In other words, $\boldsymbol{D}$ consists of both the input matrix $\boldsymbol{X}$ and target vector $y$, where: 

$\boldsymbol{X} =\left[\begin{array}{ccccc} 
  x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \cdots & x_m^{(1)}\\ 
  x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \cdots & x_m^{(2)}\\
  x_1^{(3)} & x_2^{(3)} & x_3^{(3)} & \cdots & x_m^{(3)}\\
  \vdots    & \vdots    & \vdots    & \cdots & \vdots \\
  x_1^{(n)} & x_2^{(n)} & x_3^{(n)} & \cdots & x_m^{(n)}
\end{array}\right]$

and

$\boldsymbol{y} =\left[\begin{array}{c} 
  y^{(1)}\\ 
  y^{(2)}\\
  y^{(3)}\\
  \vdots \\
  y^{(n)}
\end{array}\right]$

For unsupervised learning, $\boldsymbol{D}$ is the same as $\boldsymbol{X}$. Here is a class named `DataSet` to represent datasets. It uses pandas' DataFrame.

Another name for $X$ is `inputs`, and another name for $y$ is `target`. In addition, features have names. Let's put all of this together in a class that we will be using in subsequent weeks.

In [1]:
import numpy as np
import pandas as pd

#testing this repository

class DataSet:
    """
    A dataset for a machine learning problem. A dataset d has the following properties:
    d.examples   A list of examples. Each one contains both the features and the target.
    d.features   An array of the of feature names.
    d.target     An m by 1 array containing the values of y
    d.y          Same as d.target
    d.inputs     An n by m array containing the values of X
    d.X          Same as d.inputs
    d.N          Number of examples
    d.M          Number of dimensions
    d.name       The name of the data set (for output display only)
    
    """
    def __init__(self, data, features=None, y=None, name=None):
        """
        If y is True, the data contains the target as the last column
        If y is None or False, No target is available
        Else y is an array to be added as the last column of the examples  dataframe
        """
        self.__name = name
        if isinstance(data, pd.DataFrame):
            self.__examples = data
        else:
            self.__examples = pd.DataFrame(data, columns=features)
            
        if y is True:
            self.__examples.columns = [*self.__examples.columns[:-1], 'y']
        elif y is not False and y is not None:
            self.__examples['y'] = y
    
    @property
    def examples(self):
        return self.__examples
    
    @property
    def features(self):
        return self.__examples.columns[:-1].values
    
    @property
    def target(self):
        if 'y' in self.__examples.columns:
            return self.__examples['y'].values.reshape(self.N, 1)
        return None
    
    @property
    def y(self):
        return self.target
    
    @property
    def inputs(self):
        return self.__examples.iloc[:, :-1].values
    
    @property
    def X(self):
        return self.inputs
    
    @property
    def name(self):
        return self.__name
    
    @property
    def N(self):
        return self.__examples.shape[0]
    
    @property
    def M(self):
        return self.inputs.shape[1]
    
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
    
    def train_test_split(self, start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
    def train_validation_test_split(self, training=.5, validation=.25, test = .25, shuffle = False):
        
        deltaOne=round(self.N*training) #get numerical amount of training data
        deltaTwo=round(self.N*validation) #get numerical amount of validation group
                                         #the remaining amount will make up the test data group; 
                                         #not needed but can be insinuated; = self.N-round(self.N*training)-round(self.N*validation)
        if (deltaOne+deltaTwo>self.N): #catch extremely rare but possible rounding error.
            raise TypeError("rounding has caused critical error, should only happen when selecting miniscul sections.  Raise the decimal values and try again")

        Interim, Td = self.train_test_split(start=0, end = deltaOne, shuffle=shuffle) #grab the training data Td
        Te, Vd = Interim.train_test_split(end= deltaTwo, shuffle=shuffle) #split the remaining into Validation data Vd, and test data Te

        return Td, Vd, Te



    def __repr__(self):
        return repr(self.examples)

This class has a couple of properties including `name` (informational), `features` (the names of the features), `inputs`, `target`, `X`, `y`, `N` (number of examples), `M` (number of dimensions).

A DataSet object is created using a NumPy array or a Pandas dataframe. If it is a NumPy array, the class uses it to create a Pandas dataframe. The dataframe storing the data can be retrieved back using the `examples` property.


Let's test this class by creating a $27 \times 3$ input data and a separate $y$ column.

In [2]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")

ds

     x1   x2         x1  y
0   7.0  6.0  10.926358  1
1   8.0  2.0  10.851477  1
2   5.0  1.0  11.509140  0
3   2.0  1.0   8.290084  0
4   3.0  2.0  12.378069  0
5   5.0  5.0  10.938747  0
6   2.0  7.0  12.924745  1
7   5.0  2.0   7.192158  1
8   4.0  6.0   9.768400  1
9   7.0  2.0   6.881212  0
10  5.0  6.0   7.798990  0
11  2.0  5.0  11.508276  1
12  7.0  6.0  12.491060  1
13  2.0  4.0   9.022107  0
14  7.0  1.0  13.438968  0
15  3.0  1.0   7.859804  1
16  3.0  2.0  11.348273  1
17  5.0  6.0   7.031212  1
18  3.0  7.0   8.929119  0
19  2.0  2.0   8.496868  1
20  3.0  1.0  10.364991  1
21  4.0  1.0  10.714218  0
22  2.0  3.0   7.291712  1
23  8.0  2.0  10.621138  0
24  3.0  6.0  12.646149  0
25  7.0  8.0  10.144261  0
26  2.0  3.0  11.165869  0

In [3]:
ds.examples

,x1,x2,x1,y
0,7.0,6.0,10.926358,1
1,8.0,2.0,10.851477,1
2,5.0,1.0,11.509140,0
3,2.0,1.0,8.290084,0
4,3.0,2.0,12.378069,0
5,5.0,5.0,10.938747,0
6,2.0,7.0,12.924745,1
7,5.0,2.0,7.192158,1
8,4.0,6.0,9.768400,1
9,7.0,2.0,6.881212,0


In [4]:
ds.features

array(['x1', 'x2', 'x1'], dtype=object)

In [5]:
ds.target 

array([[1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0]])

In [6]:
ds.y 

array([[1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0]])

In [7]:
ds.inputs

array([[ 7.        ,  6.        , 10.92635813],
       [ 8.        ,  2.        , 10.85147709],
       [ 5.        ,  1.        , 11.5091397 ],
       [ 2.        ,  1.        ,  8.29008382],
       [ 3.        ,  2.        , 12.37806895],
       [ 5.        ,  5.        , 10.93874677],
       [ 2.        ,  7.        , 12.92474496],
       [ 5.        ,  2.        ,  7.19215838],
       [ 4.        ,  6.        ,  9.76840038],
       [ 7.        ,  2.        ,  6.88121198],
       [ 5.        ,  6.        ,  7.7989904 ],
       [ 2.        ,  5.        , 11.50827624],
       [ 7.        ,  6.        , 12.49106006],
       [ 2.        ,  4.        ,  9.02210743],
       [ 7.        ,  1.        , 13.43896754],
       [ 3.        ,  1.        ,  7.85980399],
       [ 3.        ,  2.        , 11.34827331],
       [ 5.        ,  6.        ,  7.0312117 ],
       [ 3.        ,  7.        ,  8.92911914],
       [ 2.        ,  2.        ,  8.49686785],
       [ 3.        ,  1.        , 10.364

In [8]:
ds.X

array([[ 7.        ,  6.        , 10.92635813],
       [ 8.        ,  2.        , 10.85147709],
       [ 5.        ,  1.        , 11.5091397 ],
       [ 2.        ,  1.        ,  8.29008382],
       [ 3.        ,  2.        , 12.37806895],
       [ 5.        ,  5.        , 10.93874677],
       [ 2.        ,  7.        , 12.92474496],
       [ 5.        ,  2.        ,  7.19215838],
       [ 4.        ,  6.        ,  9.76840038],
       [ 7.        ,  2.        ,  6.88121198],
       [ 5.        ,  6.        ,  7.7989904 ],
       [ 2.        ,  5.        , 11.50827624],
       [ 7.        ,  6.        , 12.49106006],
       [ 2.        ,  4.        ,  9.02210743],
       [ 7.        ,  1.        , 13.43896754],
       [ 3.        ,  1.        ,  7.85980399],
       [ 3.        ,  2.        , 11.34827331],
       [ 5.        ,  6.        ,  7.0312117 ],
       [ 3.        ,  7.        ,  8.92911914],
       [ 2.        ,  2.        ,  8.49686785],
       [ 3.        ,  1.        , 10.364

In [9]:
ds.name

'Sample Data'

In [10]:
ds.N

27

In [11]:
ds.M

3

## Shuffling

The above class also supports a few useful methods. One such method is for shuffling the data, which we do often before training. This method returns a new DataSet instance with the shuffled data. Here is how this method is implemented:

```python
    ...
    def shuffled(self, random_state=None):
        rgen = np.random.RandomState(random_state)
        indexes = np.arange(self.N)
        rgen.shuffle(indexes)
        return DataSet(self.__examples.iloc[indexes])
   ...
```

Here is an example using this function.

In [12]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ds.shuffled()

     x1   x2         x1  y
20  6.0  8.0   9.667436  0
26  3.0  5.0  10.413521  0
16  6.0  7.0  10.086967  0
7   5.0  3.0   9.951592  1
4   4.0  1.0  12.150181  1
13  5.0  6.0  11.744329  1
18  6.0  2.0  12.069277  0
17  5.0  4.0  13.565921  0
5   4.0  7.0  11.612853  1
0   2.0  1.0   7.987676  1
8   4.0  4.0  10.454254  0
9   3.0  1.0  12.516565  0
22  4.0  6.0   7.022800  1
24  7.0  1.0  10.740198  0
25  2.0  8.0   7.361384  0
2   3.0  5.0  11.825453  1
14  5.0  2.0  11.374971  0
3   2.0  6.0  12.358657  1
15  7.0  4.0  13.085227  0
6   6.0  4.0   7.991209  1
11  4.0  8.0  10.782843  0
12  2.0  7.0  12.097148  0
23  3.0  5.0  10.168530  1
19  6.0  1.0   9.866206  1
21  6.0  6.0   9.324946  0
1   4.0  1.0   9.756266  0
10  6.0  8.0  10.195245  1

## Splitting a dataset into training and test datasets

Another useful method provided by the above dataset class is the `train_test_split` method. This method splits the dataset into a training and test sets. Here is how this method is implemented:

```python
    ...
    def train_test_split(self,start=0, end=None, test_portion=None, shuffle=False, random_state=None):
        """
        Splits the dataset into a training set and atest set. 
        If test_portion is specified, return that portion of the dataset as test 
        and the rest as training. 
        Otherwise, return the examples between start and end as test and the 
        rest as training.
        """
        indexes = np.arange(self.N)
        if shuffle is True:
            rgen = np.random.RandomState(random_state)
            rgen.shuffle(indexes)

        if test_portion is None:
            end = end or self.N
        else:
            if not isinstance(test_portion, float) or test_portion < 0 or test_portion > 1:
                raise TypeError("Only fractions between ]0,1[ are allowed")

            start = self.N - int(self.N * test_portion)
            end = self.N

        test = DataSet(self.examples.iloc[indexes[range(start, end)]])
        train = DataSet(pd.concat([self.examples.iloc[indexes[range(start)]], 
                                      self.examples.iloc[indexes[range(end, self.N)]]], axis=0))    
        return train, test
   ...
```

If the `start` and end `end` parameters exist, the method returns the examples before them as test and the rest of the data as training. If `test_portion` is provided, then that portion of the data is returned as test and the rest as training. The `shuffle` parameter can be used to instruct the method to shuffle the data before splitting it. The method finally returns two dataset instances: training and test sets.

Here is an example using this method.

In [13]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   8.0  6.0   7.498641  0
1   5.0  1.0   8.643289  0
2   3.0  1.0   8.611536  1
3   2.0  3.0  11.424597  1
4   6.0  1.0   9.654239  0
5   7.0  2.0   8.376758  1
6   2.0  7.0  14.792608  1
7   5.0  2.0  12.824355  1
8   2.0  5.0   9.760571  0
9   5.0  7.0  11.810050  1
10  6.0  2.0  12.135151  1
11  4.0  4.0   8.119080  1
12  5.0  1.0   9.484552  0
13  6.0  6.0  10.360960  1
14  8.0  4.0   9.816987  0
15  6.0  2.0  14.075299  0
16  5.0  1.0  10.394993  1
17  6.0  4.0   8.704796  0
18  7.0  7.0   7.620926  1
19  7.0  2.0  10.659908  1
20  6.0  5.0  12.004132  0
Test set = 
      x1   x2         x1  y
21  2.0  8.0  11.523020  0
22  4.0  1.0  10.960690  1
23  8.0  7.0   9.602581  1
24  8.0  5.0   6.992095  1
25  8.0  2.0  11.089145  1
26  6.0  3.0  11.146020  0


## Using this dataset class inside other notebooks

This class is part of the `mylib` library of this class with is provided to you. Here is how to import this library:

In [14]:
import mylib as my

Once imported, one can use it like this:

In [15]:
ds = my.DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, te = ds.train_test_split(test_portion=.25, shuffle=False, random_state=17)
print('Training set = \n', ta)
print('Test set = \n', te)

Training set = 
      x1   x2         x1  y
0   6.0  3.0   9.020860  0
1   6.0  6.0  10.678919  0
2   7.0  2.0  14.084885  1
3   4.0  3.0   7.450352  1
4   3.0  5.0  12.572254  1
5   5.0  8.0   7.365757  1
6   3.0  5.0   9.693847  0
7   6.0  7.0  14.276436  1
8   3.0  2.0   9.385050  1
9   4.0  8.0   8.519113  1
10  5.0  4.0   7.412229  1
11  5.0  2.0   7.901162  0
12  3.0  5.0  11.356749  0
13  4.0  7.0  10.493882  1
14  4.0  5.0  11.619002  0
15  3.0  1.0  11.144791  0
16  4.0  5.0   9.935301  1
17  6.0  4.0   8.415612  1
18  7.0  2.0   9.451154  1
19  7.0  7.0  12.742022  0
20  5.0  5.0  10.562173  1
Test set = 
      x1   x2         x1  y
21  8.0  2.0  11.360837  1
22  5.0  2.0  10.058150  1
23  4.0  1.0  12.155980  1
24  2.0  7.0   9.765551  1
25  3.0  1.0  10.910030  1
26  7.0  3.0  10.329082  0


## EXERCISE

Refactor the above DataSet class by adding a method named `train_validation_test_split` to it. This method should split the data into three sets: training, validation, and test. This method should receive a dictionary parameter named `portions` specifying how much of the data is in each set. For a 75%/15%/10% split, one can use the following portions parameter:

```python
portions={"training": .75, 'validation': .15, 'test': .10 }
```

The method should support the `shuffle` parameter as well. You may call the `train_test_split` method internally. Make sure to include a comment describing how your implementation of the method works. Test your method on the `ds` dataset above and show that it works.

In [16]:
ds = DataSet(np.array([
    np.random.randint(2,9, 27),
    np.random.randint(1,9, 27),
    np.random.normal(loc=10, scale=2, size=27)
]).T, features=['x1', 'x2', 'x1'], y=np.random.randint(0,2, 27), name="Sample Data")


ta, vd, te = ds.train_validation_test_split(training=.7, validation=.2, test=.1, shuffle = True)
print('Training set =', ta.N, '\n', ta)
print('Validation =', vd.N, '\n', vd)
print('Test set =', te.N, '\n', te)

Training set = 19 
      x1   x2         x1  y
5   4.0  8.0  11.799472  1
25  7.0  2.0  11.899816  1
10  6.0  7.0   7.163362  0
23  3.0  5.0  10.136399  1
17  7.0  6.0   9.047746  1
18  8.0  7.0  12.958982  0
0   2.0  1.0  13.485957  1
13  6.0  1.0  10.115473  1
6   8.0  2.0   8.545959  0
24  6.0  2.0   9.991838  1
9   3.0  3.0  12.199664  0
11  6.0  1.0   9.447792  0
12  4.0  2.0  10.168088  0
15  2.0  3.0  13.983070  0
14  3.0  1.0  11.316448  1
3   8.0  2.0   9.173354  0
22  5.0  2.0   9.677988  0
21  7.0  8.0   7.988673  0
2   7.0  7.0   9.690532  0
Validation = 5 
      x1   x2         x1  y
16  8.0  8.0  11.130439  0
19  6.0  3.0  11.809273  1
20  4.0  6.0  11.984272  0
4   5.0  3.0  12.654580  0
26  8.0  8.0  10.750876  0
Test set = 3 
     x1   x2         x1  y
8  8.0  3.0  10.232317  0
1  4.0  4.0   9.088414  1
7  8.0  1.0  10.821226  0
